In [1]:
import torch
from time import time
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from zipfile import ZipFile
import numpy as np
import pandas as pd
from time import time
import zipfile
import re
import os
import torchvision
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.io import read_image
from matplotlib import image

In [2]:
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from prepare_dataset import MyDataset,subset_generator
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from train_fn import train,validation,image_validation,validation_classes,train_batch
from Model import Network,new_model,new_Regressor
from utils import MyCuda_Stats

In [3]:
MyCuda_Stats()

No of GPUs i have is 1
0
My Graphic Card is NVIDIA GeForce RTX 2060 with Max-Q Design
Is Cuda Available True


In [4]:
fullvgg=full_vgg()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])
target_transform=transforms.Compose([transforms.ToTensor()])


In [5]:
#will take lot of time due to opening each file and creating the dataframe
anno_dir=r"C:\Users\jacob\Downloads\Data\train_set\annotations"
image_dir=r"C:\Users\jacob\Downloads\Data\train_set\images"
train_data=MyDataset(anno_dir,image_dir,transform=transform, target_transform=None)


This dataset is done



In [6]:

val_anno_dir=r"C:\Users\jacob\Downloads\Data\val_set\annotations"
val_image_dir=r"C:\Users\jacob\Downloads\Data\val_set\images"
val_data=MyDataset(val_anno_dir,val_image_dir,transform=transform, target_transform=None)

This dataset is done



In [7]:
batch = 4
train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch,
                                         shuffle=True,
                                         num_workers=4,
                                         pin_memory=True,
                                         sampler=None)
val_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=batch,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)
val_class_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=1,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)

In [8]:
df = train_data.basis_to_exp()
X = np.array(df.loc[:, ~df.columns.isin(['class'])])
y = np.array(df['class'])
print(X)
print(y)

[]
[]


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
feature_extractor=vgg_layer_freeze(17)
total_trainable_parameters(feature_extractor)


Total trainable parameters of Sequential is 	 12979200


In [11]:

#input_features_for_denselayer=get_output_features_of_model(model=feature_extractor,batch_size=32,dataloader=train_loader)
input_features_for_denselayer=25088


In [12]:
modelReg=new_Regressor(feature_extractor,input_features_for_denselayer).to(device)
total_trainable_parameters(modelReg)


Total trainable parameters of new_Regressor is 	 117842434


In [13]:
loss_plain=nn.L1Loss()
optimizer=optim.NAdam(modelReg.parameters(),lr=0.0001)
classes = ['Valance', 'Arousal']
classes_exp=['Neutral','Happy','Sad','Surprise','Fear','Disgust','Anger','Contempt']

In [ ]:
epochs = 5
for epoch in range(epochs):
    print(f'Epoch-{epoch}')
    train_batch(train_loader,modelReg,loss_plain,optimizer)
    validation(val_loader,modelReg,loss_plain)

Epoch-0


  0%|          | 0/71913 [00:00<?, ?it/s]C:\Users\jacob\anaconda3\envs\dl_env\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
  0%|          | 1/71913 [00:13<279:14:56, 13.98s/it]

loss:0.078901 batch:0/71913
loss:0.023218 batch:0/71913


  0%|          | 3/71913 [00:14<76:13:56,  3.82s/it] 

loss:0.127345 batch:2/71913
loss:0.168670 batch:2/71913


...

loss:0.000652 batch:984/71913
loss:0.000619 batch:984/71913


  1%|▏         | 987/71913 [04:53<5:33:52,  3.54it/s]

loss:0.000015 batch:986/71913
loss:0.000023 batch:986/71913


  1%|▏         | 989/71913 [04:53<5:36:05,  3.52it/s]

loss:0.000024 batch:988/71913
loss:0.000023 batch:988/71913


In [ ]:
from tqdm import tqdm
def train12(dataloader,model,loss_fn,optimizer):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.train()
    for batch,(X,y_exp,y_val,y_aro) in enumerate(tqdm(dataloader)):
        y_val = y_val.type(torch.LongTensor).view(-1,1)
        y_aro = y_aro.type(torch.LongTensor).view(-1,1)
        X,y_val,y_aro=X.to(device),y_val.to(device),y_aro.to(device)

        predVal, predAro = model(X)
        print(predVal.shape)
        print(predAro.shape)
        print(y_val.shape)
        print(y_aro.shape)
        break
        """loss1=loss_fn(pred,predVal)
        loss2=loss_fn(pred,predAro)

        optimizer.zero_grad()
        loss1.backward()
        loss2.backward()
        optimizer.step()
        """
train12(train_loader,modelReg,loss_plain,optimizer)

In [ ]:
"""
Trains the val and arousal to give the emotion class
The depth is limited to 6 to prevent overfitting and on recognizing complexity from human perspective
The training is independant of the NN but will take the output of the NN to get the emotion class
The dataset X, y will take a lot of time due to the 
"""

from xgboost import XGBRFClassifier

# using random forest due to visible relation
clf = XGBRFClassifier(objective='multi:softmax')

n_estimators = [10,100, 200]
learning_rate = [0.1, 1]
booster = ['dart']
tree_method = ['hist']
eval_metric = ['auc']

gamma = [0.1, 1, 5]
#l1
reg_alpha = [0.1,1,10]
#l2 lambda required
reg_lambda = [0.1,1,10]
n_jobs = [-1]
subsample = [0.33]
max_depth = [6]


grid_state = {'n_estimators':n_estimators,
              'learning_rate':learning_rate,
              'booster':booster , 
              'tree_method':tree_method,
              'n_jobs':n_jobs,
              'eval_metric':eval_metric,
              'reg_alpha':reg_alpha,
              'reg_lambda':reg_lambda,
              'gamma': gamma,
              'subsample':subsample,
              'max_depth':max_depth
             }

xgb_grid = GridSearchCV(estimator = clf,
                        param_grid = grid_state,
                        scoring = "roc_auc",
                        cv=5,
                        verbose=0,
                    )
xgb_grid.fit(X,y)

In [ ]:
def predict(clf, reg):
    # to write once someone has the NN model
    